In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-preprocessing/__results__.html
/kaggle/input/train-preprocessing/__notebook__.ipynb
/kaggle/input/train-preprocessing/__output__.json
/kaggle/input/train-preprocessing/Train_Preprocess.csv
/kaggle/input/train-preprocessing/custom.css
/kaggle/input/testpre/Test_Preprocess.csv


In [2]:
print(os.getcwd() )
train_df=pd.read_csv("/kaggle/input/train-preprocessing/Train_Preprocess.csv")

test_df=pd.read_csv("/kaggle/input/testpre/Test_Preprocess.csv")

/kaggle/working


TfidfVectorizer

.fit(raw_documents[, y])     Learn vocabulary and idf from training set.


.fit_transform(raw_documents[, y])  Learn vocabulary and idf, return document-term matrix.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics

#tf idf
tf_idf = TfidfVectorizer(max_features=20000, ngram_range = (1,2))  #max_features=21000
X_train_tf = tf_idf.fit_transform(train_df['review_text_preprocess'].apply(lambda x: np.str_(x)))

test_tfidf = tf_idf.transform(test_df['review_text_preprocess'].apply(lambda x: np.str_(x)))

In [ ]:
# train_df.describe()
X_train_tf

In [ ]:
len(pd.unique(train_df['book_id']))
# len(pd.unique(test_df['book_id']))

In [ ]:
from sklearn.preprocessing import OneHotEncoder

train_df['book_id'] = train_df['book_id'].astype('category') 
test_df['book_id'] = test_df['book_id'].astype('category')


# Create an instance of One-hot-encoder 
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_df[['book_id']])
train_enc_data = enc.transform(train_df[['book_id']])
test_enc_data = enc.transform(test_df[['book_id']])

# Passing encoded columns  
# train_enc_data = pd.DataFrame(enc.fit_transform(train_df[['book_id']]).toarray()) 

In [ ]:
print(train_enc_data.shape)
print(test_enc_data.shape)

In [ ]:
from scipy.sparse import hstack
train = hstack((X_train_tf, train_enc_data))
test = hstack((test_tfidf, test_enc_data))

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=2000, C=0.1)
model.fit(train, train_df['rating'])

In [ ]:
# Make predictions on the test set
y_pred = model.predict(test)

sub = pd.DataFrame()
sub['review_id']=test_df['review_id']
sub['rating']=y_pred

sub.to_csv("bookid_logistic_feature_20000_Not_balanced_C_0-1.csv")
print("logistic_feature_20000_Not_balanced_C_1_.csv complete")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight

model = LogisticRegression(max_iter=2000, C=0.5)
model.fit(train, train_df['rating'])

# Make predictions on the test set
y_pred = model.predict(test)

sub = pd.DataFrame()
sub['review_id']=test_df['review_id']
sub['rating']=y_pred

sub.to_csv("bookid_logistic_feature_20000_Not_balanced_C_0-5.csv")
print("logistic_feature_20000_Not_balanced_C_1_.csv complete")

In [ ]:
#c=0.1
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=1)

model.fit(X_train_tf, train_df['rating'])

# Make predictions on the test set
y_pred = model.predict(test)

sub = pd.DataFrame()
sub['review_id']=test_df['review_id']
sub['rating']=y_pred

sub.to_csv("bookid_logistic_feature_20000_balanced_C_1.csv")
print("logistic_feature_20000_Not_balanced_C_1_.csv complete")

In [ ]:
#c=0.1
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=0.1)

model.fit(X_train_tf, train_df['rating'])

# Make predictions on the test set
y_pred = model.predict(test)

sub = pd.DataFrame()
sub['review_id']=test_df['review_id']
sub['rating']=y_pred

sub.to_csv("bookid_logistic_feature_20000_balanced_C_0-1.csv")
print("logistic_feature_20000_Not_balanced_C_1_.csv complete")

In [ ]:
#c=0.1
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=0.5)

model.fit(X_train_tf, train_df['rating'])

# Make predictions on the test set
y_pred = model.predict(test)

sub = pd.DataFrame()
sub['review_id']=test_df['review_id']
sub['rating']=y_pred

sub.to_csv("bookid_logistic_feature_20000_balanced_C_0-5.csv")
print("logistic_feature_20000_Not_balanced_C_1_.csv complete")

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# from sklearn.utils.class_weight import compute_class_weight

# class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=1)

# model.fit(X_train_tf, train_df['rating'])



In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_20000_balanced_C_1.csv")
# print("logistic_feature_20000_balanced_C_1_.csv complete")


In [ ]:
# #c=0.1
# class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=0.1)

# model.fit(X_train_tf, train_df['rating'])

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_20000_balanced_C_0-1.csv")
# print("logistic_feature_20000_balanced_C_0-1_.csv complete")

In [ ]:
# #c=0.01
# class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=0.01)

# model.fit(X_train_tf, train_df['rating'])

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_20000_balanced_C_0-01.csv")
# print("logistic_feature_20000_balanced_C_0-1_.csv complete")

In [6]:
# model = LogisticRegression(max_iter=2000, C=1)
# model.fit(X_train_tf, train_df['rating'])

LogisticRegression(C=0.5, max_iter=2000)

In [8]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_20000_Not_balanced_C_1.csv")
# print("logistic_feature_20000_Not_balanced_C_1_.csv complete")

logistic_feature_20000_Not_balanced_C_1_.csv complete


In [ ]:
# model = LogisticRegression(max_iter=2000, C=0.1)
# model.fit(X_train_tf, train_df['rating'])

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_20000_Not_balanced_C_0-1.csv")
# print("logistic_feature_20000_Not_balanced_C_0-1.csv complete")

In [ ]:
# model = LogisticRegression(max_iter=2000, C=0.01)
# model.fit(X_train_tf, train_df['rating'])

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_20000_Not_balanced_C_0-01.csv")
# print("logistic_feature_20000_Not_balanced_C_0-01.csv complete")

In [ ]:
# print(os.getcwd() )
# train_df=pd.read_csv("/kaggle/input/train-preprocessing/Train_Preprocess.csv")
# test_df=pd.read_csv("/kaggle/input/testpre/Test_Preprocess.csv")

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# from sklearn import metrics
# train_df['review_text_preprocess'] = train_df['review_text_preprocess'].apply(lambda x: np.str_(x))

# #tf idf
# tf_idf = TfidfVectorizer(ngram_range = (1,2))
# X_train_tf = tf_idf.fit_transform(train_df['review_text_preprocess'])

# test_df['review_text_preprocess']=test_df['review_text_preprocess'].apply(lambda x: np.str_(x))
# test_tfidf = tf_idf.transform(test_df['review_text_preprocess'])

# print("TfidfVectorizer Complete")

In [ ]:
# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(max_iter=2000, C=1)
# model.fit(X_train_tf, train_df['rating'])
# print("trainig Complete")

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_No_Limit_Not_balanced_C_1.csv")
# print("logistic_feature_No_Limit_balanced_C_0-01.csv complete")

In [ ]:
# #c=1
# class_weights = compute_class_weight('balanced', classes=np.unique(train_df['rating']), y=train_df['rating'])
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs',class_weight=dict(zip(np.unique(train_df['rating']), class_weights)),max_iter=2000, C=1)

# model.fit(X_train_tf, train_df['rating'])

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(test_tfidf)

# sub = pd.DataFrame()
# sub['review_id']=test_df['review_id']
# sub['rating']=y_pred

# sub.to_csv("logistic_feature_No_Limit_balanced_C_1.csv")
# print("logistic_feature_No_Limit_balanced_C_0-01.csv complete")